In [ ]:
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Access folder in Google Drive
folder_path = '/content/drive/MyDrive/Colab Notebooks/Binary Depression Analysis'
os.chdir(folder_path)

Mounted at /content/drive


In [ ]:
!pip install pandas
!pip install nltk

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from collections import Counter, defaultdict

# Download stop words
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Binary Depression Analysis/datasets/merged_d1_and_d2.csv")

In [ ]:
df.head()

,text,label
0,@hoemiceder also the same is with my depressio...,depressed
1,who the fuck let me watch the story of my life...,depressed
2,@Atypical_LDS I have been diagnosed with depre...,depressed
3,@markiplier hey I love that you did this chari...,depressed
4,@remove32 I have been diagnosed with depressio...,depressed


In [ ]:
# Remove rows where the 'text' column is NaN
df.dropna(subset=['text'], inplace=True)

In [ ]:
# Initialize an empty dictionary to hold frequencies for each word with respect to labels
word_frequencies = defaultdict(lambda: defaultdict(int))

# List of stop words to be removed
stop_words = set(stopwords.words('english'))

# Populate the frequency dictionary
for index, row in df.iterrows():
    words = nltk.word_tokenize(row['text'].lower())
    words = [w for w in words if w.isalnum() and w not in stop_words]  # Remove punctuation and stopwords
    for word in words:
        word_frequencies[word][row['label']] += 1

# Convert frequency data to a form that can inform about depression-related words
depression_lexicon = {}
for word, label_freq in word_frequencies.items():
    depression_score = label_freq.get('depressed', 0)
    not_depressed_score = label_freq.get('not_depressed', 0)
    if depression_score + not_depressed_score > 0:
        depression_lexicon[word] = depression_score / (depression_score + not_depressed_score)

print("Depression Lexicon:", depression_lexicon)

Depression Lexicon: {'hoemiceder': 1.0, 'also': 0.7484276729559748, 'depression': 0.9998430634023855, 'diagnosed': 1.0, 'clinical': 1.0, 'luckily': 1.0, 'control': 0.6666666666666666, 'fuck': 0.5877192982456141, 'let': 0.4485981308411215, 'watch': 0.2708333333333333, 'story': 0.6818181818181818, 'life': 0.7624113475177305, 'music': 0.54, 'video': 0.05699481865284974, 'agree': 0.4583333333333333, '100': 0.625, 'sad': 0.7964601769911505, 'markiplier': 0.9285714285714286, 'hey': 0.5714285714285714, 'love': 0.20987654320987653, 'charity': 0.75, 'severe': 1.0, 'lt': 0.5352112676056338, '3': 0.7277486910994765, 'amazing': 0.5681818181818182, 'remove32': 1.0, 'gp': 0.9230769230769231, 'medicated': 1.0, 'seeing': 0.6, 'psychiatrist': 1.0, 'soon': 0.3902439024390244, 'official': 0.52, 'diagnosis': 1.0, 'louis': 0.5, 'luv': 0.5714285714285714, 'u': 0.463768115942029, 'much': 0.5541125541125541, 'syndrome': 0.96, 'called': 0.6285714285714286, 'post': 0.9064748201438849, 'concert': 0.9574468085106

#JSON

In [ ]:
import json

with open("depression_lexicon.json", "w") as f:
    json.dump(depression_lexicon, f)


In [ ]:
with open("depression_lexicon.json", "r") as f:
    loaded_lexicon = json.load(f)


## CSV

In [ ]:
import csv

with open("depression_lexicon.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Word", "Frequency"])
    for key, value in depression_lexicon.items():
        writer.writerow([key, value])


In [ ]:
import pandas as pd

loaded_lexicon_df = pd.read_csv("depression_lexicon.csv")
loaded_lexicon = dict(zip(loaded_lexicon_df.Word, loaded_lexicon_df.Frequency))
